In [4]:
# !pip install equinox

In [5]:
"""
new dqn
"""

import math
from typing import Any, Callable, Dict
import jax
import equinox as eqx
from equinox import nn
from jax import random
import jax.numpy as jnp

from typing import Any, Dict, Tuple
import jax
from jax import vmap, numpy as jnp
from functools import partial

# import memory.ffa as ffa
def leaky_relu(x, key=None):
    return jax.nn.leaky_relu(x, negative_slope=0.01)

def default_init(key, linear, scale=1.0, zero_bias=False, fixed_bias=None):
    lim = math.sqrt(scale / linear.in_features)
    linear = eqx.tree_at(lambda l: l.weight, linear, jax.random.uniform(key, linear.weight.shape, minval=-lim, maxval=lim))
    if zero_bias:
        linear = eqx.tree_at(lambda l: l.bias, linear, jnp.zeros_like(linear.bias))
    elif fixed_bias is not None:
        linear = eqx.tree_at(lambda l: l.bias, linear, jnp.full_like(linear.bias, fixed_bias))
    return linear

def final_linear(key, input_size, output_size, scale=0.01):
    #linear = ortho_linear(key, input_size, output_size, scale=scale)
    linear = nn.Linear(input_size, output_size, key=key)
    linear = default_init(key, linear, scale=scale)
    linear = eqx.tree_at(lambda l: l.bias, linear, linear.bias * 0.0)
    return linear

class CNN(eqx.Module):
    c0: nn.Conv2d
    ln0: nn.LayerNorm
    c1: nn.Conv2d
    ln1: nn.LayerNorm
    c2: nn.Conv2d
    ln2: nn.LayerNorm
    linear: nn.Linear

    def __init__(self, output_size, in_channels=3, *, key):
        keys = random.split(key, 4)
        self.c0 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4, key=keys[0])
        self.ln0 = nn.LayerNorm((63, 63), use_weight=False, use_bias=False)
        self.c1 = nn.Conv2d(32, 64, kernel_size=4, stride=2, key=keys[1])
        self.ln1 = nn.LayerNorm((30, 30), use_weight=False, use_bias=False)
        self.c2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, key=keys[2])
        self.ln2 = nn.LayerNorm((28, 28), use_weight=False, use_bias=False)
        self.linear = nn.Linear(64 * 28 * 28, output_size, key=keys[3])

    def __call__(self, x, keys=None):
        # Move channels to first dim [C, W, H]
        if x.ndim == 3:
            x = jnp.moveaxis(x, -1, 0)
            # print(x.shape)

        x = leaky_relu(eqx.filter_vmap(self.ln0)(self.c0(x)))
        x = leaky_relu(eqx.filter_vmap(self.ln1)(self.c1(x)))
        x = leaky_relu(eqx.filter_vmap(self.ln2)(self.c2(x)))

        x = self.linear(x.flatten())
        return x


key = random.PRNGKey(0)
config = {
    "mlp_size": 512,
    "num_channels": 3,
    "recurrent_size": 512,
    "dropout": 0.5,
    "ensemble_size": 50,
    "cnn": True
}
# value = config.get("cnn")
# print(value)
key = jax.random.PRNGKey(0)
keys = jax.random.split(key, 10)

model = CNN(output_size=512, key=keys[1])
x = jnp.ones((256, 256, 3))
y = model(x)
print(y.shape)

model = eqx.filter_vmap(CNN(output_size=config["mlp_size"], key=keys[1]))
x_batch = jnp.ones((10, 256, 256, 3))
y = model(x_batch)
print(y.shape)


(512,)
(10, 512)


In [6]:
class RandomSequential(nn.Sequential):
    def __call__(self, x, key=None):
        return super().__call__(x, key=key)

class Block(eqx.Module):
    net: eqx.Module
    def __init__(self, input_size, output_size, dropout, key):
        if dropout == 0.0:
            self.net = RandomSequential([
                nn.Linear(input_size, output_size, key=key),
                nn.LayerNorm(output_size, use_weight=False, use_bias=False),
                leaky_relu,
            ])
        else:
            self.net = RandomSequential([
                nn.Linear(input_size, output_size, key=key),
                nn.LayerNorm(output_size, use_weight=False, use_bias=False),
                nn.Dropout(dropout),
                leaky_relu,
            ])

    def __call__(self, x, key=None):
        return self.net(x, key=key)

class QHead(eqx.Module):
    post0: eqx.Module
    post1: eqx.Module
    value: nn.Linear
    advantage: nn.Linear

    def __init__(self, input_size, hidden_size, output_size, dropout, key):
        keys = random.split(key, 3)

        self.post0 = eqx.filter_vmap(Block(input_size, hidden_size, dropout, keys[0]))
        self.post1 = eqx.filter_vmap(Block(hidden_size, hidden_size, dropout, keys[1]))
        self.value = eqx.filter_vmap(final_linear(keys[2], input_size, 1, scale=0.01))
        self.advantage = eqx.filter_vmap(final_linear(keys[3], input_size, output_size, scale=0.01))

    def __call__(self, x, key):
        T = x.shape[0]
        net_keys = random.split(key, 2 * T)
        x = self.post0(x, net_keys[:T])
        x = self.post1(x, net_keys[T:2*T])
        V = self.value(x)
        A = self.advantage(x)
        # Dueling DQN
        return V + (A - A.mean(axis=-1, keepdims=True))

class QNetwork(eqx.Module):
    input_size: int
    output_size: int
    config: Dict[str, Any]
    pre: eqx.Module
    # memory: eqx.Module
    q: eqx.Module

    def __init__(self, obs_shape, act_shape, config, key):
        self.config = config
        self.output_size = act_shape
        keys = random.split(key, 4)
        if config.get("cnn"):
            self.input_size = (256, 256, 3)
            self.pre = eqx.filter_vmap(CNN(config["mlp_size"], key=keys[1]))
        else:
            [self.input_size] = obs_shape
            self.pre = eqx.filter_vmap(Block(self.input_size, config["mlp_size"], 0, keys[1]))
        # self.memory = memory_module

        ensemble_keys = random.split(keys[0], config["ensemble_size"])

        @eqx.filter_vmap
        def make_heads(key):
            return QHead(config["recurrent_size"], config["mlp_size"], act_shape, config["dropout"], key)

        self.q = make_heads(ensemble_keys)


    def __call__(self, x, key):
        T = x.shape[0]
        net_keys = random.split(key, T + 1)
        x = self.pre(x, net_keys[:T])
        print(f"x.shape after CNN is: {x.shape}")
        # y, final_state = self.memory(x=x, state=state, start=start, next_done=done, key=key)

        @eqx.filter_vmap(in_axes=(eqx.if_array(0), None, None))
        def ensemble(model, x, key):
            return model(x, key=key)


        q = ensemble(self.q, x, net_keys[-1])
        return q

    def evaluate(self, x, state, start, done, key):
        q = self(x, state, start, done, key)
        return jnp.median(q, axis=0)

    def initial_state(self, shape=tuple()):
        return self.memory.initial_state(shape)

obs_shape = 512
act_shape = 5
model_key = jax.random.PRNGKey(0)
q_network = QNetwork(obs_shape, act_shape, config, model_key)
# q_target = eqx.tree_inference(QNetwork(obs_shape, act_shape, config, model_key), True)

# test
x_batch = jnp.ones((10, 256, 256, 3))
y = q_network(x_batch, model_key)
print(y.shape)



x.shape after CNN is: (10, 512)
(50, 10, 5)
